 # 実験3（抽象構文木の利用）
 - 前回に続き抽象構文木に対する処理を学ぶ．
 - 例として後置記法（逆ポーランド記法）への変換や抽象構文木の描画を扱う．

 ## 抽象構文木の変換

 以下は前回と同じ（ライブラリの読み込みと数式を切り出すパーサ`expr`の定義）

In [ ]:
from toyparsing import *

num = pat("[0-9]+") > int

@parser_do
def factor(run):
    return run(num | w("(") >> expr << w(")"))

@parser_do
def term(run):
    a = run(factor)
    while True:
        r = run(pat("[*/]") & factor, nullable=True) 
        if r is None: break
        [op, b] = r
        a = [op, a, b]
    return a

@parser_do
def expr(run):
    a = run(term)
    while True:
        r = run(pat("[+-]") & term, nullable=True) 
        if r is None: break
        [op, b] = r
        a = [op, a, b]
    return a

 ## 抽象構文木から後置記法（逆ポーランド記法）への変換
 普通の数式は2項演算子をふたつの被演算子の中間に置いて`1+2`のように書く．これを**中置記法**と呼ぶ．
 それにたいして`+ 1 2`のように演算子を前に置く記法を**前置記法**（あるいはポーランド記法），
 `1 2 +`のように後に置く記法を**後置記法**（あるいは逆ポーランド記法）と呼ぶ．
 抽象構文木(AST)を後置記法の式になおすには，木を深さ優先探索し，葉から根に戻るときに演算子を最後に出力すればよい．
 以下のような非常に簡単なプログラムになる．

In [ ]:
def postfix(ast):
    if isinstance(ast, int):
        return str(ast)
    [op, ast_left, ast_right] = ast         
    return postfix(ast_left) + " " + postfix(ast_right) + " " + op

In [ ]:
compile_to_postfix = expr > postfix

In [ ]:
compile_to_postfix("1+2*3")    

In [ ]:
compile_to_postfix("1-2-3-4")

In [ ]:
compile_to_postfix("1-(2-(3-4))")

In [ ]:
compile_to_postfix("1+2*3-4/5+7")

 ### [課題3]
 抽象構文木（べき乗は含まれないとしてよい）をスタック機械の命令列になおす関数codeを作成せよ．
 スタック機械の命令は以下の5つである．
 (1) PUSH 数 (数をスタックに積む)
 (2) ADD     (スタックのトップの2つの数をそれらの加算結果で置き換える)
 (2) SUB     (スタックのトップの2つの数をそれらの減算結果で置き換える)
 (2) MUL     (スタックのトップの2つの数をそれらの乗算結果で置き換える)
 (2) DIV     (スタックのトップの2つの数をそれらの除算結果で置き換える)
 具体例は下記の実行例のコメントを参照せよ．
 [ヒント] 上記の`postfix`が参考になる．

In [ ]:
compile_to_code = expr > code

In [ ]:
compile_to_code("1+2*3")  # ('PUSH 1; PUSH 2; PUSH 3; MUL; ADD; ', '')

In [ ]:
compile_to_code("1-2-3-4")  # ('PUSH 1; PUSH 2; SUB; PUSH 3; SUB; PUSH 4; SUB; ', '')

In [ ]:
compile_to_code("1-(2-(3-4))")  # ('PUSH 1; PUSH 2; PUSH 3; PUSH 4; SUB; SUB; SUB; ', '')

In [ ]:
compile_to_code("1+2*3-4/5+7")  # ('PUSH 1; PUSH 2; PUSH 3; MUL; ADD; PUSH 4; PUSH 5; DIV; SUB; PUSH 7; ADD; ',
 '')

 ## Graphvizを用いた抽象構文木の描画
 Pythonから[Graphvis](https://www.graphviz.org/gallery/)を用いて抽象構文木を描画する．

 ライブラリの読み込み
 [注意]
  実験で配布したSingularityコンテナにはgraphvizが導入済みです．
  この代わりに自分で独自に用意したJupyter Notebook環境で実験するときには
  graphvizの`dot`コマンドとPython3の`graphviz`モジュールが必要です．
  以下はMacとUbuntuでの導入例です．

  MacでHomebrewとpipを用いてインストールする場合
  ```
  brew install graphviz
  sudo pip3 install graphviz
  ```

  Ubuntuでaptパッケージとpipを用いてインストールする場合
  ```
  sudo apt install graphv
  sudo pip3 install graphviz
  ```

In [ ]:
from graphviz import Digraph

 ### graphvizの簡単な使い方
 有効グラフ（digraph)の土台(g)をつくり，それに頂点(node) n1, n2, n3と有向辺(edge)を追加していく．

In [ ]:
g = Digraph()
g.node("n1", "1")
g.node("n2", "1")
g.node("n3", "+")
g.edge("n3","n1")
g.edge("n3","n2")

 最後にグラフの土台(g)を評価するとJupyterノートブックに直接描画される（ファイルに保存することも可能）
 詳しい使い方： [公式マニュアル](https://graphviz.readthedocs.io/en/stable/manual.html)

In [ ]:
g

 ### [課題4]
 与えられた抽象構文木をGraphvizを用いて描画する関数`draw_ast`をつくれ．
 `draw_ast`は引数として抽象構文木`ast`と最初の頂点の名前を表す文字列`id`を取る．

 [ヒント] 重複の無い頂点の新しい名前を生成しながら処理を進めるのがポイント．
 そこで最初に与えられた頂点の名前の末尾に適当な文字列（例えば"L","R"）を接尾辞として付け足して
 新しい頂点の名前を作り出すとよい．

In [ ]:
@parser_do
def draw_expr(run):
    global g # 次の代入ではグローバル変数`g`に代入
    g = Digraph()
    draw_ast(run(expr), "root")

In [ ]:
draw_expr("1+2*3")
g

In [ ]:
draw_expr("(1+2)*3")
g

In [ ]:
draw_expr("1-2-3-4")
g

In [ ]:
draw_expr("1-(2-(3-4))")
g

 実験レポート作成のために描画した抽象構文木をPDFファイルとして保存したい場合は
```
 g.render(PDFファイル名)
```
 とすればよい．例えば，`g.render("ast01")`とするとノートブックと同じディレクトリに`ast01.pdf`というPDFファイルが出来上がる．